ABOUT THE DATASET

Context.

This dataset is one out of 8 collection of datasets from different sources related to the automatic detection of cyber-bullying. The data is from different social media platforms like Kaggle, Twitter, Wikipedia Talk pages and YouTube.

Content. The data contains different types of cyber-bullying, on this case we are going to analyze toxicity. The data contain text and labeled as bullying or not.

the previous information about the dataset was taken from the dataset website source: https://www.kaggle.com/datasets/saurabhshahane/cyberbullying-dataset?resource=download, and adapted to our project proposal.

In [ ]:
!pip install kaggle emoji ekphrasis symspellpy spacy nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 1.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.0 MB/s eta 0:00:00
  Created wheel for editdistpy: filename=editdistpy-0.1.4-cp310-cp310-linux_x86_64.whl size=187486 sha256=59c7873b3041ce997b1eddfa13d0575948a9a241f79cf863c0d6adbda43fceff
  Stored in directory: /root/.cache/pip/wheels/4c/0f/10/c20d67cd765ee5b3666d759a307241bba0663135d6ee1c0072
Successfully built editdistpy


In [ ]:
#Import libraries

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from collections import Counter
import matplotlib.pyplot as plt
import emoji
from emoji import demojize
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from google.colab import drive
drive.mount('/content/drive')



nltk.download('stopwords')
nltk.download('wordnet')

Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#Reading from Kaggle API and unzipping the file locally

import pandas as pd
from io import BytesIO
from zipfile import ZipFile
import os

# Upload Kaggle API Credentials (in colab)
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jaisy198","key":"8b0724b74264a22caa0ecd579c970d95"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import kaggle

In [ ]:
#Load the dataset

#df = pd.read_csv('ssamplenlp.csv') #This is here to read the file locally, we created a 800 records sample from the original file

dataset = 'saurabhshahane/cyberbullying-dataset'
file_name = 'toxicity_parsed_dataset.csv'

# Download the file content into memory
download_path = 'cyberbullying-dataset'

# Download the dataset files and unzip them into the specified path
kaggle.api.dataset_download_files(dataset, path=download_path, unzip=True)


# Construct the file path
file_path = os.path.join(download_path, file_name)

# Load the specific file into a pandas DataFrame
df = pd.read_csv(file_path)

print(df.head())

Dataset URL: https://www.kaggle.com/datasets/saurabhshahane/cyberbullying-dataset
   index                                               Text  ed_label_0  \
0      0  This: :One can make an analogy in mathematical...         0.9   
1      1  `  :Clarification for you  (and Zundark's righ...         1.0   
2      2                          Elected or Electoral? JHK         1.0   
3      3  `This is such a fun entry.   Devotchka  I once...         1.0   
4      4  Please relate the ozone hole to increases in c...         0.8   

   ed_label_1  oh_label  
0         0.1         0  
1         0.0         0  
2         0.0         0  
3         0.0         0  
4         0.2         0  


EXPLORING THE DATASET

In [ ]:
#Data description

print(df.describe())

          ed_label_0     ed_label_1       oh_label
count  159686.000000  159686.000000  159686.000000
mean        0.854951       0.145049       0.096201
std         0.253866       0.253866       0.294868
min         0.000000       0.000000       0.000000
25%         0.800000       0.000000       0.000000
50%         1.000000       0.000000       0.000000
75%         1.000000       0.200000       0.000000
max         1.000000       1.000000       1.000000


In [ ]:
#Dataset shape

print(df.shape)

(159686, 5)


In [ ]:
#Data info

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159686 entries, 0 to 159685
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   index       159686 non-null  int64  
 1   Text        159686 non-null  object 
 2   ed_label_0  159686 non-null  float64
 3   ed_label_1  159686 non-null  float64
 4   oh_label    159686 non-null  int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 6.1+ MB
None


In [ ]:
#Label distribution

print(df['oh_label'].value_counts())

oh_label
0    144324
1     15362
Name: count, dtype: int64


In [ ]:
#Unique values

df.nunique()

index         159686
Text          159388
ed_label_0        66
ed_label_1        66
oh_label           2
dtype: int64

In [ ]:
#Print text samples of each label

print('_________Text Samples_________')
sample_tweet_0 = df[df['ed_label_0'] == 1]['Text'].iloc[0]
print('\nSample No Toxic Tweet: ', sample_tweet_0)
sexist_tweet = df[df['ed_label_1'] == 1]['Text'].iloc[0]
print('\n\nSample Toxic Tweet: ', sexist_tweet)
print('\n_______________________________')

_________Text Samples_________

Sample No Toxic Tweet:  `  :Clarification for you  (and Zundark's right, i should have checked the Wikipedia bugs page first).   This is a ``bug`` in the code that makes wikipedia work  it just means that there is a line of code that may have an error as small as an extra space. It's analogous (in a VERY simplified way) to trying to make something bold in HTML and forgetting to put the  at the end, so you'd see something like this:  words in bold  Instead of this:  words in bold   It's not like a virus, that is code somebody deliberately wrote in order to infect your computer and damage files, so it won't ``go around.`` JHK   `


Sample Toxic Tweet:  i have a dick, its bigger than yours! hahaha

_______________________________


**Data** **Preprocessing**


In [ ]:
#Drop index column

df = df.drop(['index'], axis=1)

Data Cleaning

In [ ]:
#Look for missing values

print(df.isnull().any())
print(df.isnull().sum())

Text          False
ed_label_0    False
ed_label_1    False
oh_label      False
dtype: bool
Text          0
ed_label_0    0
ed_label_1    0
oh_label      0
dtype: int64


**Step 1: Handle non-grammatical/digital symbols**

In [ ]:
# Define function to clean text
def text_cleaning(text):
        #Remove Special Chars
    text = re.sub(r'[^a-zA-Z\s]', '', text)
        # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
        # Remove IP addresses
    text = re.sub(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', '', text)
        # Remove mentions (@username)
    text = re.sub(r'@\w+', '', text)
        # Remove hashtags
    text = re.sub(r'#\w+', '', text)
        # Remove digits
    text = re.sub(r'\d+', '', text)
        # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

**Step** **2: Handle slangs/emojis/abbreviatures**

In [ ]:
 #Emoji handling
def handle_emojis(text):
    return demojize(text)

# Setting up ekphrasis
text_processor = TextPreProcessor(
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user', 'time', 'date', 'number'],
    annotate={"hashtag", "allcaps", "elongated", "repeated", 'emphasis', 'censored'},
    fix_html=True,
    segmenter="twitter",
    corrector="twitter",
    unpack_hashtags=True,
    unpack_contractions=True,
    spell_correct_elong=False,
    tokenizer=SocialTokenizer(lowercase=False).tokenize,
    dicts=[emoticons]
)

# Comprehensive preprocessing
def preprocess_text(text):
    text = handle_emojis(text)
    text = ' '.join(text_processor.pre_process_doc(text))
    return text

/usr/local/lib/python3.10/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.10/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [ ]:
#Apply functions to text

df['Text'] = df['Text'].apply(text_cleaning)
df['Text'] = df['Text'].apply(handle_emojis)
df['Text'] = df['Text'].apply(preprocess_text)

In [ ]:
print(df['Text'])

0         This One can make an analogy in mathematical t...
1         Clarification for you and Zundarks right i sho...
2             Elected or Electoral <allcaps> JHK </allcaps>
3         This is such a fun entry Devotchka I once had ...
4         Please relate the ozone hole to increases in c...
                                ...                        
159681    These sources dont exactly exude a sense of im...
159682    The Institute for Historical Review is a peerr...
159683    The way youre trying to describe it in this ar...
159684    Warning There is clearly a protectionist regim...
159685    Alternate option Is there perhaps enough newsw...
Name: Text, Length: 159686, dtype: object


**Step 3-NER**

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 57.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
from spacy import displacy

# Load the English language model
nlp = spacy.load('en_core_web_sm')

In [ ]:
def perform_ner(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return text, entities

**Step 4 -Spell Check**

In [ ]:
from symspellpy import SymSpell, Verbosity
import pkg_resources

# Initialize SymSpell object
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Load dictionary
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt"
)
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

True

In [ ]:
def spell_check(text):
    if not isinstance(text, str):
        return text  # Return the input as is if it's not a string

    words = text.split()
    corrected_words = []
    for word in words:
        if word.isupper() or not word.isalpha():
            corrected_words.append(word)  # Preserve acronyms and special characters
        else:
            # Perform spell checking
            suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
            if suggestions:
                corrected_word = suggestions[0].term
            else:
                corrected_word = word
            # Preserve the original case of the word
            if word[0].isupper():
                corrected_word = corrected_word.capitalize()
            corrected_words.append(corrected_word)
    return ' '.join(corrected_words)


Preprocess Text with NER and Spell Check

In [ ]:
def preprocess_text_with_ner_and_spell_check(text): #1hr50mins
    text = preprocess_text(text)
    text_with_ner, entities = perform_ner(text) #Perform Named Entity Recognition (NER) on the preprocessed text
    text_with_spell_check = spell_check(text_with_ner) # Perform spell check on the text with NER annotations
    final_text_with_ner, final_entities = perform_ner(text_with_spell_check) #Perform NER again on the spell-checked text to ensure entity recognition is accurate
    return final_text_with_ner, final_entities

df['preprocessed_text'], df['entities'] = zip(*df['Text'].apply(preprocess_text_with_ner_and_spell_check))
print(df[['Text', 'preprocessed_text', 'entities']].head())

                                                Text  \
0  This One can make an analogy in mathematical t...   
1  Clarification for you and Zundarks right i sho...   
2      Elected or Electoral <allcaps> JHK </allcaps>   
3  This is such a fun entry Devotchka I once had ...   
4  Please relate the ozone hole to increases in c...   

                                   preprocessed_text  \
0  This One can make an analogy in mathematical t...   
1  Clarification for you and Zundarks right i sho...   
2  Elected or Electoral <allcaps> <allcaps> JHK <...   
3  This is such a fun entry Devotchka I once had ...   
4  Please relate the ozone hole to increases in c...   

                                            entities  
0  [(Gaussian, NORP), (three, CARDINAL), (Locke R...  
1  [(Zundarks, PERSON), (Wikipedia, ORG), (JHK, O...  
2                                       [(JHK, ORG)]  
3  [(Korea, GPE), (Usaenglish, ORG), (British, NO...  
4                                [(Cd Door, PERSON)

**Step 5-Run PoS Tagger**

In [ ]:
def pos_tag(text):
    # Process the text with spaCy
    doc = nlp(text)

    # Extract words and their PoS tags
    pos_tags = [(token.text, token.pos_) for token in doc]

    # Join the words and tags into a single string
    tagged_text = ' '.join([f"{word}_{tag}" for word, tag in pos_tags])

    return tagged_text, pos_tags

# Function to apply PoS tagging to our dataframe
def apply_pos_tagging(df, text_column='Text'):
    # Apply PoS tagging
    df['tagged_text'], df['pos_tags'] = zip(*df[text_column].apply(pos_tag))
    return df

# Apply PoS tagging to our dataframe
df = apply_pos_tagging(df)

# Display a few examples
print(df[['Text', 'tagged_text', 'pos_tags']].head())

                                                Text  \
0  This One can make an analogy in mathematical t...   
1  Clarification for you and Zundarks right i sho...   
2      Elected or Electoral <allcaps> JHK </allcaps>   
3  This is such a fun entry Devotchka I once had ...   
4  Please relate the ozone hole to increases in c...   

                                         tagged_text  \
0  This_DET One_PRON can_AUX make_VERB an_DET ana...   
1  Clarification_NOUN for_ADP you_PRON and_CCONJ ...   
2  Elected_VERB or_CCONJ Electoral_PROPN <_PROPN ...   
3  This_PRON is_AUX such_DET a_DET fun_NOUN entry...   
4  Please_INTJ relate_VERB the_DET ozone_NOUN hol...   

                                            pos_tags  
0  [(This, DET), (One, PRON), (can, AUX), (make, ...  
1  [(Clarification, NOUN), (for, ADP), (you, PRON...  
2  [(Elected, VERB), (or, CCONJ), (Electoral, PRO...  
3  [(This, PRON), (is, AUX), (such, DET), (a, DET...  
4  [(Please, INTJ), (relate, VERB), (the, DET), (..

**Step 6. Handle Contractions, Negations**

In [ ]:
# Dictionary of contractions
contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'll": "he will",
    "he's": "he is",
    "i'd": "I would",
    "i'll": "I will",
    "i'm": "I am",
    "isn't": "is not",
    "it's": "it is",
    "let's": "let us",
    "mightn't": "might not",
    "mustn't": "must not",
    "shan't": "shall not",
    "she'd": "she would",
    "she'll": "she will",
    "she's": "she is",
    "shouldn't": "should not",
    "that's": "that is",
    "there's": "there is",
    "they'd": "they would",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "we'd": "we would",
    "we're": "we are",
    "weren't": "were not",
    "we've": "we have",
    "what'll": "what will",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "where's": "where is",
    "who'd": "who would",
    "who'll": "who will",
    "who're": "who are",
    "who's": "who is",
    "who've": "who have",
    "won't": "will not",
    "wouldn't": "would not",
    "you'd": "you would",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have"
}

def expand_contractions(text):
    def replace(match):
        return contractions[match.group(0).lower()]

    # Use case-insensitive regex to find contractions
    pattern = re.compile(r'\b(?:%s)\b' % '|'.join(contractions.keys()), flags=re.IGNORECASE)
    return pattern.sub(replace, text)

def handle_negations(text):
    # Add a special token for negations
    negation_pattern = re.compile(r'\b(?:not|no|never|none|nobody|nowhere|neither|nor|nothing)\b', flags=re.IGNORECASE)
    return negation_pattern.sub(r'NEG_\g<0>', text)

    # Apply the functions to the 'Text' column
df['Text'] = df['Text'].apply(expand_contractions)
df['Text'] = df['Text'].apply(handle_negations)

**Step 7-Use regex to handle punctuation/symbols, etc**

In [ ]:
def clean_text_with_regex(text):
    # Replace URLs with a token
    text = re.sub(r'https?://\S+|www\.\S+', '[URL]', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Replace repeated punctuation with single punctuation
    text = re.sub(r'([!?.]){2,}', r'\1', text)
    # Replace repeated characters (e.g., 'goooood' to 'good')
    text = re.sub(r'(\w)\1{2,}', r'\1\1', text)
    # Separate punctuation from words (e.g., "word!" to "word !")
    text = re.sub(r'([a-z])([!?.])', r'\1 \2', text)
    # Remove all other punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Strip leading and trailing whitespace
    text = text.strip()

    return text

# Apply to our dataframe
df['cleaned_text'] = df['Text'].apply(clean_text_with_regex)

**Step 8 -Tokenize**

In [ ]:
def custom_tokenize(text):
    # Split on whitespace
    tokens = text.split()

    # Handle hashtags, mentions, and emojis
    processed_tokens = []
    for token in tokens:
        if token.startswith('#'):
            processed_tokens.extend(['#', token[1:]])
        elif token.startswith('@'):
            processed_tokens.append('[USER]')
        elif token in ['[URL]', '[EMAIL]']:
            processed_tokens.append(token)
        else:
            # Split token on punctuation, keeping the punctuation
            processed_tokens.extend(re.findall(r"\w+|[^\w\s]", token))

    return processed_tokens

# Apply custom tokenization
df['custom_tokens'] = df['cleaned_text'].apply(custom_tokenize)

In [ ]:
df.head()

,Text,ed_label_0,ed_label_1,oh_label,preprocessed_text,entities,tagged_text,pos_tags,cleaned_text,custom_tokens
0,This One can make an analogy in mathematical t...,0.9,0.1,0,This One can make an analogy in mathematical t...,"[(Gaussian, NORP), (three, CARDINAL), (Locke R...",This_DET One_PRON can_AUX make_VERB an_DET ana...,"[(This, DET), (One, PRON), (can, AUX), (make, ...",This One can make an analogy in mathematical t...,"[This, One, can, make, an, analogy, in, mathem..."
1,Clarification for you and Zundarks right i sho...,1.0,0.0,0,Clarification for you and Zundarks right i sho...,"[(Zundarks, PERSON), (Wikipedia, ORG), (JHK, O...",Clarification_NOUN for_ADP you_PRON and_CCONJ ...,"[(Clarification, NOUN), (for, ADP), (you, PRON...",Clarification for you and Zundarks right i sho...,"[Clarification, for, you, and, Zundarks, right..."
2,Elected or Electoral <allcaps> JHK </allcaps>,1.0,0.0,0,Elected or Electoral <allcaps> <allcaps> JHK <...,"[(JHK, ORG)]",Elected_VERB or_CCONJ Electoral_PROPN <_PROPN ...,"[(Elected, VERB), (or, CCONJ), (Electoral, PRO...",Elected or Electoral allcaps JHK allcaps,"[Elected, or, Electoral, allcaps, JHK, allcaps]"
3,This is such a fun entry Devotchka I once had ...,1.0,0.0,0,This is such a fun entry Devotchka I once had ...,"[(Korea, GPE), (Usaenglish, ORG), (British, NO...",This_PRON is_AUX such_DET a_DET fun_NOUN entry...,"[(This, PRON), (is, AUX), (such, DET), (a, DET...",This is such a fun entry Devotchka I once had ...,"[This, is, such, a, fun, entry, Devotchka, I, ..."
4,Please relate the ozone hole to increases in c...,0.8,0.2,0,Please relate the ozone hole to increases in c...,"[(Cd Door, PERSON)]",Please_INTJ relate_VERB the_DET ozone_NOUN hol...,"[(Please, INTJ), (relate, VERB), (the, DET), (...",Please relate the ozone hole to increases in c...,"[Please, relate, the, ozone, hole, to, increas..."


In [ ]:
#save df to Google Drive

df.to_csv('cleaned_df.csv', index=False)

In [ ]:
!cp /content/cleaned_df.csv /content/drive/MyDrive/NLPProject